### Notebook 3 - technology graph description (solution 2)

In [1]:
from networkx.readwrite import json_graph
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
import warnings
import json
import os

warnings.filterwarnings('ignore')
year_start = 1975
year_end = 1999
graph_folder = '/Users/koshelev/Documents/lmu/thesis/1-graph_construction/data/preprocessed/technology_graphs_2/'
weight = 'Number Citations'
seed = 100

In [2]:
def read_json_file(filename: str) -> nx.Graph:
    with open(filename) as f:
        js_graph = json.load(f)
    return json_graph.node_link_graph(js_graph)

In [3]:
def graph_algebraic_connectivity(graph: nx.DiGraph, 
                                 is_normalized: bool = False, 
                                 weight_attribute: str = None, 
                                 seed: int = seed) -> float:
    alg_con = nx.linalg.algebraic_connectivity(G=graph, normalized=is_normalized, weight=weight_attribute, 
                                               seed=seed)
    return alg_con


def graph_wiener_index(graph: nx.DiGraph, weight_attribute: str = weight) -> float:
    wiener_index = nx.wiener_index(G=graph, weight=weight_attribute)
    return wiener_index

In [4]:
def graph_descr(graph: nx.DiGraph, weight: str) -> dict:
    output_dict = {}
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()
    average_degree_w = np.nanmean(list(dict(graph.degree(weight=weight)).values()))
    average_degree_nw = np.nanmean(list(dict(graph.degree(weight=None)).values()))
    average_lcc_nw = nx.average_clustering(G=graph, weight=None)
    try:
        average_spl_nw = nx.average_shortest_path_length(G=graph, weight=None)
    except nx.NetworkXError:
        average_spl_nw = np.nan
    algebraic_connect_nw = graph_algebraic_connectivity(graph=graph.to_undirected(), weight_attribute=None)
    average_deg_centr = np.mean(list(nx.degree_centrality(G=graph).values()))
    
    output_dict['num_nodes'] = num_nodes
    output_dict['num_edges'] = num_edges
    output_dict['average_degree_w'] = average_degree_w
    output_dict['average_degree_nw'] = average_degree_nw
    output_dict['average_lcc_nw'] = average_lcc_nw
    output_dict['average_spl_nw'] = average_spl_nw
    output_dict['algebraic_connect_nw'] = algebraic_connect_nw
    output_dict['average_deg_centr'] = average_deg_centr
    
    return output_dict

In [5]:
nodes = []
edges = []
degrees_w = []
degrees_nw = []
alcc_nw = []
spl_nw = []
algebrcon_nw = []
deg_centr = []
years = []

for year in tqdm(range(year_start, year_end + 1)):
    g = read_json_file(graph_folder + f'technology_graph_{int(year)}.json')
    print(year)
    years.append(year)
    output_dict = graph_descr(g, weight=weight)
    nodes.append(output_dict['num_nodes'])
    edges.append(output_dict['num_edges'])
    degrees_w.append(output_dict['average_degree_w'])
    degrees_nw.append(output_dict['average_degree_nw'])
    alcc_nw.append(output_dict['average_lcc_nw'])
    spl_nw.append(output_dict['average_spl_nw'])
    algebrcon_nw.append(output_dict['algebraic_connect_nw'])
    deg_centr.append(output_dict['average_deg_centr'])
    
descr_df = pd.DataFrame()
descr_df['nodes'] = nodes
descr_df['edges'] = edges
descr_df['degrees_w'] = degrees_w
descr_df['degrees_nw'] = degrees_nw
descr_df['alcc_nw'] = alcc_nw
descr_df['spl_nw'] = spl_nw
descr_df['algebrcon_nw'] = algebrcon_nw
descr_df['deg_centr'] = deg_centr
descr_df.index = years

  0%|          | 0/25 [00:00<?, ?it/s]

1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999


In [6]:
descr_df.to_csv('graph_descriptions/technology2.csv', index=True)

In [7]:
descr_df.head()

,nodes,edges,degrees_w,degrees_nw,alcc_nw,spl_nw,algebrcon_nw,deg_centr
1975,92,848,4675.543478,18.434783,0.696479,1.380674,0.905301,0.202580
1976,92,874,4954.478261,19.000000,0.736404,1.263856,0.932424,0.208791
1977,97,884,4602.474227,18.226804,0.736185,1.281250,0.931246,0.189863
1978,90,888,5210.177778,19.733333,0.714217,1.355556,0.590353,0.221723
1979,89,820,4074.674157,18.426966,0.713620,1.228677,0.941732,0.209397
